In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
train_path='/content/drive/MyDrive/BuildingForest/Train_2'
test_path='/content/drive/MyDrive/BuildingForest/Test_2'

In [ ]:
import os
print(len(os.listdir('/content/drive/MyDrive/BuildingForest/Train_2')))
print(len(os.listdir('/content/drive/MyDrive/BuildingForest/Test_2')))
print(len(os.listdir('/content/drive/MyDrive/BuildingForest/Train_2/Desert')))
print(len(os.listdir('/content/drive/MyDrive/BuildingForest/Train_2/Urban')))
print(len(os.listdir('/content/drive/MyDrive/BuildingForest/Test_2/Dessert')))
print(len(os.listdir('/content/drive/MyDrive/BuildingForest/Test_2/Urban')))

2
2
411
420
113
100


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator (
    rescale = 1./255,
    validation_split = 0.2
)

train_data = train_datagen.flow_from_directory (
    train_path,
    target_size = (256, 256),
    batch_size = 32,
    class_mode = 'binary',
    subset = 'training'
)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # Note: Using the same validation split as in the training set
)

validation_data = validation_datagen.flow_from_directory(
    train_path,
    target_size=(256,256),
    batch_size=32,
    class_mode='binary',
    subset='validation'  # Specify 'validation' for the validation set
)


Found 665 images belonging to 2 classes.
Found 166 images belonging to 2 classes.


In [ ]:
print(train_data.class_indices)


{'Desert': 0, 'Urban': 1}


In [ ]:
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,UpSampling2D, Dropout

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

# Load ResNet50 model without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze the layers of ResNet to keep the pre-trained weights
base_model.trainable = False

# Build the model
model = Sequential([
    base_model,  # Add the base ResNet50 model
    GlobalAveragePooling2D(),  # Global Average Pooling layer to reduce the spatial dimensions
    Dense(256, activation='relu'),  # Fully connected layer
    Dropout(0.3),  # Dropout to prevent overfitting
    Dense(128, activation='relu'),  # Fully connected layer
    Dropout(0.3),  # Dropout to prevent overfitting
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary to check layers
model.summary()


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 8, 8, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         524,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 24,145,281 (92.11 MB)

 Trainable params: 557,569 (2.13 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
# Train the model
history = model.fit(
    train_data,  # Training data
    epochs=8,  # Number of epochs (you can adjust as needed)
    validation_data=validation_data  # Validation data
)


Epoch 1/8
21/21 ━━━━━━━━━━━━━━━━━━━━ 281s 13s/step - accuracy: 0.5861 - loss: 0.6687 - val_accuracy: 0.9759 - val_loss: 0.2933
Epoch 2/8
21/21 ━━━━━━━━━━━━━━━━━━━━ 188s 9s/step - accuracy: 0.9111 - loss: 0.2924 - val_accuracy: 0.9759 - val_loss: 0.1157
Epoch 3/8
21/21 ━━━━━━━━━━━━━━━━━━━━ 192s 9s/step - accuracy: 0.9782 - loss: 0.1011 - val_accuracy: 0.9759 - val_loss: 0.1142
Epoch 4/8
21/21 ━━━━━━━━━━━━━━━━━━━━ 190s 9s/step - accuracy: 0.9779 - loss: 0.0777 - val_accuracy: 0.9578 - val_loss: 0.1178
Epoch 5/8
21/21 ━━━━━━━━━━━━━━━━━━━━ 187s 9s/step - accuracy: 0.9948 - loss: 0.0350 - val_accuracy: 0.9759 - val_loss: 0.0589
Epoch 6/8
21/21 ━━━━━━━━━━━━━━━━━━━━ 190s 9s/step - accuracy: 0.9974 - loss: 0.0166 - val_accuracy: 0.9759 - val_loss: 0.0572
Epoch 7/8
21/21 ━━━━━━━━━━━━━━━━━━━━ 188s 9s/step - accuracy: 0.9981 - loss: 0.0112 - val_accuracy: 0.9759 - val_loss: 0.0739
Epoch 8/8
21/21 ━━━━━━━━━━━━━━━━━━━━ 227s 11s/step - accuracy: 0.9997 - loss: 0.0094 - val_accuracy: 0.9759 - val_los

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)  # Normalizing test data

test_data = test_datagen.flow_from_directory(
    test_path,
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary'  # binary because we are doing Desert vs Urban classification
)


Found 213 images belonging to 2 classes.


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Get true labels and predicted labels
true_labels = test_data.classes  # Ground truth labels from test data
predictions = model.predict(test_data)  # Predictions on the test data

# Apply threshold (0.5 for binary classification)
predicted_labels = (predictions > 0.5).astype(int)

# Generate confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Print confusion matrix
print("Confusion Matrix:")
print(cm)

# Print classification report
print("Classification Report:")
print(classification_report(true_labels, predicted_labels))


7/7 ━━━━━━━━━━━━━━━━━━━━ 64s 9s/step
Confusion Matrix:
[[64 49]
 [48 52]]
Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.57      0.57       113
           1       0.51      0.52      0.52       100

    accuracy                           0.54       213
   macro avg       0.54      0.54      0.54       213
weighted avg       0.54      0.54      0.54       213



In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load an image for testing
img_path = '/content/drive/MyDrive/BuildingForest/Test_2/Urban/buildings_000398.png'  # Replace with your image path
img = image.load_img(img_path, target_size=(256, 256))  # Resize image
img_array = image.img_to_array(img)  # Convert image to array
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array /= 255.0  # Normalize image

# Predict on the image
prediction = model.predict(img_array)

# Interpret the result
if prediction[0][0] > 0.5:
    print("Urban Zone Detected")
else:
    print("Desert Zone Detected")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 369ms/step
Urban Zone Detected


In [ ]:
# Save the model
model.save('/content/drive/MyDrive/BuildingForest/desert_urban_model.h5')
print("Model saved successfully!")


Model saved successfully!


In [ ]:
# Load the saved model
from tensorflow.keras.models import load_model

model = load_model('/content/drive/MyDrive/BuildingForest/desert_urban_model.h5')
print("Model loaded successfully!")


Model loaded successfully!


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load an image for testing
img_path = '/content/drive/MyDrive/BuildingForest/Test_2/Dessert/Kazam_screenshot_00000.png'  # Replace with your image path
img = image.load_img(img_path, target_size=(256, 256))  # Resize image
img_array = image.img_to_array(img)  # Convert image to array
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array /= 255.0  # Normalize image

# Predict on the image
prediction = model.predict(img_array)

# Interpret the result
if prediction[0][0] > 0.5:
    print("Urban Zone Detected")
else:
    print("Desert Zone Detected")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Desert Zone Detected
